# Наивный Байесовский классификатор

Самый простой, часто использующийся и при этом один из самых элегантных и эффективных алгоритмов классификации — Наивный Байесовский классификатор (НБК). Он:

* Устойчив к незначимым признакам, так как просто игнорирует их
* Быстро обучается и быстро возвращает предсказание
* Потребляет относительно небольшое число ресурсов
* Показывает один из лучших результатов в задаче анализа тональности текста

Наивный Байесовский классификатор основан на теореме Байеса о вероятности события. Наивность его заключается в предположении, что все анализируемые признаки независимы, это помогает упростить работу классификатора. На практике чаще всего бывает наоборот — все признаки так или иначе влияют друг на друга. Однако "наивный" алгоритм всё равно показывает хорошие результаты, и это главное.

Ниже мы подробнее изучим основу алгоритма - саму теорему Байеса и предположение, означающее "наивность" классификатора, а затем рассмотрим его результаты на конкретном примере с классификацией твитов.

## Теорема Байеса

Суть Наивной Байесовской классификации — отслеживание того, какой признак о каком классе свидетельствует. А то, какие признаки и каким образом рассматриваются, зависит от модели обучения классификатора. 

Например, в **модели Бернулли** допускаются только булевские признаки (то есть признаки типа есть/нет). При анализе текстов эта модель будет анализировать, встречается определённое слово в тексте или нет, но не будет обращать внимание на то, **сколько раз** оно встречается. А в **мультиномиальной модели** признаками являются счетчики слов, то есть модель будет не только замечать наличие слова в тексте, но и считать, как много раз оно было использовано.

Для объяснения работы НБК при анализе тональности текста рассмотрим _ _модель Бернулли_ _, как более простую. Позже, когда мы перейдём к реальным классификаторам и задачам, будем использовать _ _мультиномиальную модель_ _.

Итак, пусть:
* Класс твита по тональности (положительный или отрицательный);
* Признак 1: в твите хотя бы раз встречается слово good;
* Признак 2: в твите хоть раз встречается слово bad.

В процессе обучения на уже размеченном по тональности корпусе твитов алгоритм строит наивную Байесовскую модель, которая для каждого твита возвращает вероятность его принадлежности к определённому классу, если известны значения обоих признаков.

Так как мы не можем посмотреть внутрь модели и посчитать вероятность принадлежности твита к определённому классу напрямую, то по формуле Байеса мы можем высчитать её через вероятность класса самого по себе и вероятность наличия в твите слов-признаков.

В виде формулы это выглядит примерно как:

$$probability = \frac{class \times likelihood}{features}$$

$class$ и $features$ легко посчитать:
* $class$ - вероятность класса без каких-либо знаний о данных. Оценить её можно напрямую подсчитав долю обучающих примеров, принадлежащих данному классу.
* $features$ - вероятность одновременного наличия слов-признаков в проверяемом твите.

Проблемы возникают при вычислении likelihood - вероятности наличия в твите определённого класса обоих слов-признаков. Но здесь в дело вступает второй важный компонент Наивного Байесовского классификатора - его "наивность".

## Предположение о наивности

Поиск вероятности одновременного наличия нескольких признаков в тексте, принадлежащем определённому классу - задача достаточно сложная, так как признаки становятся зависимы друг от друга, а "посмотреть" на их зависимость мы не можем. Мы даже не знаем, как эта зависимость выражается и есть ли она вообще.

Здесь и кроется выход: мы можем "наивно" предположить, что все признаки независимы друг от друга. Тогда для вычисления нужной вероятности надо всего лишь высчитать вероятность наличия каждого слова по отдельности в тексте определённого класса. Это сделать гораздо проще.

Собирая всё вместе, для примера с двумя словами-признаками получаем простую формулу:

$$probability = \frac{class \times good(class) \times bad(class)}{features}$$

где:

* good(class) - вероятность наличия слова-признака good в твите выбранного класса 
* bad(class) - вероятность наличия слова-признака bad в твите выбранного класса 

Любопытная вещь №1: хотя теоретически неправильно выдвигать произвольные предположения под настроение, в данном случае такой подход на удивление хорошо работает в реальных задачах.

Любопытная вещь №2: для любого количества классов знаменатель дроби - features - всегда будет одинаковым, а так как при предсказании выбирается наибольшая вероятность, то его можно проигнорировать. Наибольшее значение от этого не изменится, а формула станет проще.


## Использование наивного Байесовского алгоритма для классификации

После того, как модель обучилась на размеченнном корпусе, она может предсказывать тональность новых твитов, которых не было в корпусе для обучения.

Получив новый твит, она вычисляет вероятности его принадлежности к каждому из двух классов по формуле выше, а затем относит твит к тому классу, для которого вероятность была больше. Стоит отметить, что сами вероятности классификатор не сильно интересуют: гораздо важнее то, какой класс более правдоподобен для того или иного текста.

Проиллюстрируем, чтобы понаблюдать за работой наивного байесовского алгоритма. Сделаем предположение, что Twitter разрешает употреблять только два слова: good и bad и что мы уже вручную проклассифицировали несколько твитов:

| Твит | Класс 
| :-:  | :-:
|good | pos
|good | pos
|good bad | pos
|bad | pos
|bad | neg
|bad | neg

Твит bad получил как отрицательную, так и положительную оценку (это возможно благодаря многозначности слов и контексту).
Всего шесть твитов - 4 pos и 2 neg.

Считаем вероятности классов:
$$\Pr(Class=pos) = \frac{4}{6} ~ 0.67$$
$$\Pr(Class=neg) = \frac{2}{6} ~ 0.33$$

Это означает, что ничего не зная о самом твите, можно предположить, что он положительный. Просто их больше, но об этом мы будем говорить позже.

Пока отсутсвует вычисление вероятностей слов-признаков при условии класса. Они вычисляются как количество твитов класса, в которых встречался отдельный признак, поделенное на количество твитов, помеченных этим классом.

Вероятность встретить good, если известно что класс положительный:

$$\Pr(good \mid Class=pos) = \frac{\text{число положительных твитов, содержащих слово good}}{\text{число всех положительных твитов}} = \frac{3}{4}$$

Поскольку из 4 положительных твитов 3 содержали слово good.

Очевидно, что вероятность не встретить слово good в положительном твите равна:

$$1 - \Pr(good \mid Class=pos) = 0.25$$

Точно так же производятся остальные вычисления. 

Итоговая вероятность класса какого-либо нового твита будет зависеть от того, есть ли в нём слова good и bad. Мы умножаем вероятность класса на вероятности наличия (или отсутствия) в нём слов-признаков. Все эти компоненты уже посчитаны, и вычислить итоговые вероятности не составит труда.

Но как быть со словами не встречавшихся в тренировочном корпусе? Ведь всем новым словам будет присвоена нулевая вероятность...

## Учет ранее не встречавшихся слов

Как мы уже говорили, модель вычисляет не точные вероятности, а только правдоподобия для определения лучшего класса. Однако, используя обучающий корпус, мы предполагаем, что данные корпуса содержат полную информацию обо всех вероятностях, просто мы не используем их.

Но разве какой угодно большой корпус твитов может содержать информацию обо всех твитах, которые когда-то написаны и будут написаны? Разумеется, нет.

Если же сузить до нашего примера с шестью твитами, эту модель может ввести в ступор появление третьего слова - например, слова film. Стоит появиться таким новым твитам, и сразу станет понятно, что приблизительные вычисления нашей модели совсем не соответствуют новой реальности.

Однако учесть все слова невозможно, и эту проблему приходится обходить. Самый простой способ её решения - **сглаживание с прибавлением единицы (add-one smoothing)**.
Это очень простой приём, заключающийся в прибавлении единицы ко всем вхождениям всех признаков. В его основе лежит разумное предположение, что даже если мы не видели данного слова в обучающем корпусе, есть шанс , что это случилось только потому, что в нашей выборке таких твитов не оказалось. Однако для баланса единица прибавляется и к количеству новых признаков (чтобы избежать нулей), и к количеству старых (которое просто увеличивается на 1).

То есть вместо вычисления:

$$\Pr(good \mid Class=pos) = \frac{\text{число положительных твитов, содержащих слово good}}{\text{число всех положительных твитов}} = \frac{3}{4}$$

Мы вычисляем:

$$\Pr(good \mid Class=pos) = \frac{3+1}{4+2}=0.67$$

Почему в знаменателе прибавлено 2? 
Потому что всего у нас два признака: вхождения слов good и bad. Поскольку мы прибавляем 1 для каждого признака, нужно позаботиться, чтобы получились всё те же вероятности.

### Как построить наивный Байесовский классификатор в Python?

Наивный Байесовский классификатор уже реализован за нас в библиотеке Scikit-learn (или sklearn). Всё, что требуется - дать модели обучающие данные и данные для предсказания.

Scikit-learn предлагает 3 модели Наивного Байесовского классификатора: 

* Gaussian: предполагает, что атрибуты нормально распределены.

* Multinomial: используется для дискретных атрибутов. Например, при текстовой классификации она будет считать, сколько раз слово-признак встречается в анализируемом тексте.

* Bernoulli: модель Бернулли, оперирует бинарными параметрами типа "0 или 1". Наример, классификация текстов с моделью bag of words, где атрибуты могут быть 0 (слово не встретилось) или 1 (слово встретилось).

Ниже рассмотрен пример использования Гауссовской модели наивного Байесовского классификатора.

In [3]:
#импортируем модель из библиотеки sklearn и библиотеку numpy для работы с массивами
from sklearn.naive_bayes import GaussianNB
import numpy as np

#зададим матрицу признаков обучающих объектов и вектор значений классов, к которым они относятся
x= np.array([[-3,7],[1,5], [1,2], [-2,0], [2,3], [-4,0], [-1,1], [1,1], [-2,2], [2,7], [-4,1], [-2,7]])
Y = np.array([3, 3, 3, 3, 4, 3, 3, 4, 3, 4, 4, 4])
#создаём модель
model = GaussianNB()

# обучаем модель на заданных данных
model.fit(x, Y)

#дадим модели признаки ещё двух объектов и получим предсказанные им классы
predicted= model.predict([[1,2],[3,4]])
print(predicted)

[3 4]
